# Pre-requisites

Initialize notebook environment:

In [ ]:
import os
import sys

sys.path.append('..')
os.environ['FLASK_ENV'] = 'Development'
os.environ['MYPASS_DB_CONNECTION_URI'] = 'sqlite+pysqlite:///:memory:'

Import all neccessary libraries.

In [ ]:
import requests

## Start up flask app

You can run your flask app either here, or host on any other computer.

Here it will run as a daemonic thread in the background. You will most likely have to restart or stop the kernel to stop it.

To protect you from incidentaly running the application from here, `run_from_jupyter` flag is set to `False` by default.

In [ ]:
from threading import Thread

from mypass import create_app

run_from_jupyter = False


def main():
    app = create_app()

    host = app.config['HOST']
    port = app.config['PORT']
    app.run(host=host, port=port, debug=False)


if __name__ == '__main__':
    if run_from_jupyter:
        t = Thread(target=main, daemon=True)
        t.start()

# API usage

Set proxy for upcoming requests.

In [ ]:
proxies = {'http': 'http://localhost:5757', 'https': 'http://localhost:5757'}

In [ ]:
url = 'http://localhost/api/auth/registration'
resp = requests.post(url=url, proxies=proxies, json={'username': 'master', 'password': 'super-secret'})

if resp.status_code == 201:
    print('Successful registration!')
    resp_obj = resp.json()
    print(resp_obj)
else:
    print('Failed registration!')
    print(resp.json())

Registration will log you in at the same time, as well as save a new user in the database side.

Next, we save all the jwt we just got from the registration request.

In [ ]:
access_token = resp_obj['access_token']
refresh_token = resp_obj['refresh_token']

We can create an authorization class that does the work for us:

In [ ]:
from requests.auth import AuthBase


class BearerAuth(AuthBase):
    def __init__(self, token: str):
        self.token = token

    def __call__(self, r):
        r.headers['authorization'] = f'Bearer {self.token}'
        return r

In [ ]:
auth = BearerAuth(token=access_token)

or simply create a header object which we can pass along as well, with any given request.

In [ ]:
headers = {'authorization': f'Bearer {access_token}'}

Let's try to login, with the registered user!
You can also experiment with different, or unauthorized logins.

In [ ]:
url = 'http://localhost/api/auth/login'
resp = requests.post(url=url, proxies=proxies, json={'username': 'master', 'password': 'super-secret'})

if resp.status_code == 201:
    print('Successful login!')
    resp_obj = resp.json()
    access_token = resp_obj['access_token']
    refresh_token = resp_obj['refresh_token']
    auth = BearerAuth(token=access_token)
    headers = {'authorization': f'Bearer {access_token}'}
    print(resp_obj)
else:
    print('Failed login!')
    print(resp.json())

## Requesting non-fresh access tokens

We will attempt to request a new ***non-fresh** access token* using our previous *refresh token*.

In [ ]:
url = 'http://localhost/api/auth/refresh'
resp = requests.post(url=url, proxies=proxies, auth=BearerAuth(token=refresh_token))

if resp.status_code == 201:
    print('Successfully acquired non-fresh access token!')
    resp_obj = resp.json()
    access_token = resp_obj['access_token']
    auth = BearerAuth(token=access_token)
    headers = {'authorization': f'Bearer {access_token}'}
    print(resp_obj)
else:
    print('Failed to get new access token!')
    print(resp.json())

## Creating vault entry

Lets create a vault entry! First, we set the url, then we will be making the actual request.

In [ ]:
url = 'http://localhost/api/db/vault/add'
resp = requests.post(
    url=url,
    proxies=proxies,
    json={
        'fields': {
            'username': 'aragorn',
            'password': 'l$1swoĐf75s#d9',
            'website': 'https://favsite.com',
            'title': 'fav',
            'notes': 'Hush!'
        }
    },
    auth=BearerAuth(token=access_token))

if resp.status_code == 200:
    print('Successful entry creation!')
    resp_obj = resp.json()
    print(resp_obj)
else:
    print('Failed entry creation!')
    print(resp.json())

Let's add some more test data!

In [ ]:
url = 'http://localhost/api/db/vault/add'
test_data = [
    {'username': 'loghachi', 'password': 'ŁswndsaWI45', 'website': 'https://worldwide.com', 'title': 'world', 'folder': 'social'},
    {'username': 'hachiman', 'password': 'pa85ks546đĐä5w8S#', 'website': 'https://fakebook.com', 'title': 'fakeb', 'folder': 'social'},
    {'username': 'manner', 'password': 'aiiu3fĐđä3#', 'website': 'https://gitterhub.com', 'title': 'huby', 'folder': 'tokens'}
]

for td in test_data:
    resp = requests.post(
        url=url,
        proxies=proxies,
        json={'fields': td},
        auth=BearerAuth(token=access_token))
    
    if resp.status_code == 200:
        print(f'Added entry: {resp.json()}')
    else:
        print(resp.json())

## Reading from db

Lets query our read endpoints!

The following endpoint will read records from our database.

In [ ]:
url = 'http://localhost/api/db/vault/select'
resp = requests.post(
    url=url,
    proxies=proxies,
    json={},
    auth=BearerAuth(token=access_token))

if resp.status_code == 200:
    print('Entries read:')
    resp_obj = resp.json()
    if isinstance(resp_obj, list):
        print('\n'.join(str(o) for o in resp_obj))
    else:
        print(resp_obj)
else:
    print('Failed reading entries!')
    print(resp.json())

Next up, the update endpoint:

In [ ]:
url = 'http://localhost/api/db/vault/update'
resp = requests.post(
    url=url,
    proxies=proxies,
    json={
        'crit': {
            'folder': 'social'
        },
        'fields': {
            'notes': ':('
        }
    },
    auth=BearerAuth(token=access_token))

if resp.status_code == 200:
    print('Entries updated:')
    resp_obj = resp.json()
    print(resp_obj)
else:
    print('Failed to update entries!')
    print(resp.json())

We will delete some rows up next.

In [ ]:
url = 'http://localhost/api/db/vault/delete'
resp = requests.post(
    url=url,
    proxies=proxies,
    json={
        'crit': {
            'title': 'fakebook'
        }
    },
    auth=BearerAuth(token=access_token))

if resp.status_code == 200:
    print('Entries deleted:')
    resp_obj = resp.json()
    print(resp_obj)
else:
    print('Failed to delete entries!')
    print(resp.json())